# Assignment 2

This assignment will be concerned with demonstrating the following course contents:
* Relational databases (SQL) and MongoDB (NoSQL)
* Graph Databases
* Streaming algorithms



## SQL and NoSQL

SQL databases has over the last decades been the core norm of persisting data into non-volatile memory. Although SQL is a great way to represent certain data sets, it can be quite time expensive when queries starts to compute a high number of large cartesian products between tables. To mitigate some of these performance problems, there has been an increasing interest within the area of NoSQL (i.e. a database that is not meant to have normalized relations). MongoDB is a popular NoSQL database and provides a schema-less persistency of the data in a json-like format (called bson). In contrary to SQL, the data in a NoSQL database should be denormalized as much as possible (that is, you should store it wihin a nested structure). This makes MongoDB especially interesting when storing large documents or highly detailed log-files with a high throughput.

In this exercise we're going to use the Northwind database. To give the reader a idea of how this database looks like, from a relational perspective, a ER diagram is displayed below.

<img src="http://i3.codeplex.com/download?ProjectName=northwinddatabase&DownloadId=269240" width="800" height="800" />

The equivalent MongoDB database, that we are going to use, has onfortunately not been denormalized into one collection. Because of this, the way MongoDB is being used in this exercise is somehow _a special case_.

### Exercise 1

_Task: Establish connection to MongoDB and SQLite_

In the first exercise, we are going to make a basic connection to both SQLite (a light-weight SQL engine that persists data into files) and to MongoDB.

#### Connection to SQLite

The first thing we need to do, is to import sqlite3, which is the Python module we're going to use.

In [1]:
import sqlite3 as lite

We can create a small function that returns a connection to the northwind.db (a file in the same directory).

In [2]:
def get_sqlite_connection():
    conn = lite.connect('northwind.db')
    conn.text_factory = str
    return conn

From here on, getting a connection to the SQLite DB is easy...

In [ ]:
conn = get_sqlite_connection()

Given the `conn` object we can create a cursor, which is used to query the database.

In [ ]:
cur = conn.cursor()

Using the cursor `cur` we can can retrieve all the products by making the sql query `SELECT * FROM PRODUCTS`.

In [77]:
cur.execute('SELECT * FROM PRODUCTS')

The query has been executed and the result set can be obtained by calling fetchall. Here, we are just going to see the first element.

In [78]:
cur.fetchall()[:1] # Take the first element

[(1, 'Chai', 1, 1, '10 boxes x 20 bags', 18, 39, 0, 10, '0')]

We can query the database once again, this time, however, we are going to sort the result set using the `customerid`, which is a column in the table `CUSTOMERS`.

In [79]:
cur.execute('SELECT * FROM customers ORDER BY customerid ASC')
cur.fetchall()[:1]

[('ALFKI',
  'Alfreds Futterkiste',
  'Maria Anders',
  'Sales Representative',
  'Obere Str. 57',
  'Berlin',
  None,
  '12209',
  'Germany',
  '030-0074321',
  '030-0076545')]

#### Connection to MongoDB

Connecting to MongoDB is slightly more difficult, than connecting to a SQLite DB, since MongoDB is a client-server database engine and has to be configured.

First, we need to install MongoDB. After MongoDB is installed, we need to run a shell script that imports the northwind database into MongoDB. After this, we can start the MongoDB service on localhost and the default port using `mongod` in bash.

<img src="img/mongod-service-start.png" width="800" height="800" />

After we've done this, we can `pip install pymongo`, and we are finally ready to start interacting with Northwind MongoDB through Python.

In [5]:
from pymongo import MongoClient
import pymongo

We can create a function, that connects to MongoDB and then returns a connection to "Nortwind"

In [6]:
def get_mongo_connection():
    client=MongoClient('localhost',27017)
    return client['Northwind']   # Get the database

At this point, we can easily make return an MongoDB conn instance

In [7]:
c = get_mongo_connection()

We have established a connection and can use it to retrieve all the products and list the first entry.

In [83]:
products = c.products.find()
list(products)[0]

{u'CategoryID': 1,
 u'Discontinued': 0,
 u'ProductID': 1,
 u'ProductName': u'Chai',
 u'QuantityPerUnit': u'10 boxes x 20 bags',
 u'ReorderLevel': 10,
 u'SupplierID': 1,
 u'UnitPrice': 18.0,
 u'UnitsInStock': 39,
 u'UnitsOnOrder': 0,
 u'_id': ObjectId('5609b12899ded9537b06c7e5')}

Although, this is just a dict, it can be easily seen as a bson object, which is the original way that MongoDB persists data.

If we wanted to make yet another query, but this time we wanted to sort the result, we can apply a sort filter to a specific bson key. From here on, we're going to use Pandas DataFrame since they provide an easy way of pretty-printing tabular data.

In [84]:
c = get_mongo_connection()
customers = c.customers.find().sort('CustomerID', pymongo.ASCENDING)
top10 = [customer['CustomerID'] for customer in customers][:10]
pd.DataFrame(top10, columns=['CustomerID'])

,CustomerID
0,ALFKI
1,ANATR
2,ANTON
3,AROUT
4,BERGS
5,BLAUS
6,BLONP
7,BOLID
8,BONAP
9,BOTTM


The list above shows the top 10 customer ids, sorted in ascending order, from the customer collection.

### Exercise 2

In [ ]:
cur.execute("SELECT * FROM customers AS c INNER JOIN orders AS o ON c.customerid=o.customerid where o.customerid='ALFKI'")
pd.DataFrame(cur.fetchall())

In [ ]:
for customer in c.customers.find({'CustomerID':'ALFKI'}):
    for order in c.orders.find({'CustomerID': customer['CustomerID']}):
        for details in c['order-details'].find({'OrderID': order['OrderID']}):
            joined_result = order.copy()
            joined_result.update(details)
r = cur.fetchall()

# Pretty print using pandas
pd.DataFrame([joined_result.values()], columns=joined_result.keys())

### Exercise 3
_Get all orders (with products) made by ALFKI that contain at least 2 products._

In [15]:
import pandas as pd

In [ ]:
cur.execute('SELECT * FROM customers AS c INNER JOIN orders AS o ON c.customerid=o.customerid '
                    ' INNER JOIN "Order Details" AS od ON od.orderid=o.orderid '
                    ' INNER JOIN products AS p ON p.productid=od.productid '
                    " WHERE c.customerid='ALFKI'"
                    ' GROUP BY o.orderid HAVING count(distinct od.productid) >= 2')
r = cur.fetchall()
pd.DataFrame(r)

In [ ]:
from bson.code import Code
c = get_mongo_connection()
reducer = Code(
        """
            function(obj, prev){
              prev.count++;
            }
        """)
orders = []
for customer in c.customers.find({'CustomerID':'ALFKI'}):
    for order in c.orders.find({'CustomerID': customer['CustomerID']}):
        results = c['order-details'].group(key={"OrderID": 1}, condition={}, initial={"count": 0}, reduce=reducer)
        for details in results:
            if details['OrderID'] == order['OrderID'] and details['count'] >= 2:
                orders.append(order)
                joined_result = order.copy()
                joined_result.update(details)
                
# Pretty print using pandas
pd.DataFrame(orders)

#### Exercise 4

In [ ]:
class TestExercise54(unittest.TestCase):
    def test_sqlite_count_products(self):
        conn = connect_sqlite()
        cur = conn.cursor()
        cur.execute('SELECT count(productid) FROM "Order Details" where productid=7')
        print 'SQLite Count: ' + str(cur.fetchone()[0])

    def test_mongo_count_products(self):
        c = get_mongo_connection()
        print 'MongoDB Count: ' + str(c['order-details'].find({'ProductID': 7}).count())

#### Exercise 5

In [74]:
stmt = """
SELECT count(p.productid), p.productname FROM orders
INNER JOIN [Order Details] AS od ON od.orderid=orders.orderid
INNER JOIN products AS p ON p.productid=od.productid
WHERE CustomerID IN
            (SELECT CustomerID FROM orders
             INNER JOIN [Order Details] ON [Order Details].orderid=orders.orderid
             WHERE [Order Details].productid=7)
GROUP BY od.productid
"""
cur.execute(stmt)
#print list(cur.fetchall())
df_sql = pd.DataFrame(cur.fetchall(), columns=['count','ProductName'])
df_sql

,count,ProductName
0,8,Chai
1,18,Chang
2,7,Aniseed Syrup
3,6,Chef Anton's Cajun Seasoning
4,6,Chef Anton's Gumbo Mix
5,5,Grandma's Boysenberry Spread
6,29,Uncle Bob's Organic Dried Pears
7,5,Northwoods Cranberry Sauce
8,1,Mishi Kobe Niku
9,16,Ikura


c = get_mongo_connection()
from bson.code import Code
reducer = Code(
        """
            function(obj, prev){
              prev.count++;
            }
        """)
#results = c['order-details'].group(key={"ProductID": 1, 'CustomerID':2}, condition={}, initial={"count": 0}, reduce=reducer)
#for x in results:
#    print x
customers = {}
products = {}
for details in c['order-details'].find({'ProductID': 7}):
    for order in c.orders.find({'OrderID':details['OrderID']}):
        customers[order['CustomerID']]= 0

for customer_id in customers.keys():
    for r in c.orders.find():
        if r['CustomerID'] == customer_id:
            for details in c['order-details'].find({'OrderID':r['OrderID']}):

                product_name = c.products.find({'ProductID': details['ProductID']})[0]['ProductName']

                if product_name in products.keys():
                    products[product_name] += 1
                else:
                    products[product_name] = 1

df_mongo = pd.DataFrame(products.keys(), columns=['ProductName']).sort('ProductName')
df_mongo

## Streaming Algorithms

We use streaming algorithms when the given data is of such a size that it can not be stored at all, and where we only get to observe each sample once. Also, Wikipedia adds that *These algorithms have limited memory available to them (much less than the input size) and also limited processing time per item. These constraints may mean that an algorithm produces an approximate answer based on a summary or "sketch" of the data stream in memory.*

Three of these kind of algorithms will be examined here.

### Exercise 7.1 - Bloom filter

The Bloom filter is a probabilistic data structure, that is used to test whether an element is a member of a set. Only false positive matches can occur (the element is show as a member though it is not - "possibly in set", or "definitely not in the set"), therefore it is said that a Bloom filter has a 100% recall rate. 

In this exercise, a such filter is to be built, which is able to check the words from a text written by Shakespeare against a dictionary representing all words of the English language (*shakespeare.txt*, *dict*).

In [46]:
import numpy

class bloom_filter:
    
    # ctor of the filter
    def __init__(self, bm, hashf, hashfk):
        # initializing the bit array to zeroes
        self.__bitarray = numpy.zeros(bm)
        self.m = bm
        
        # taking the hash function. it should take two parameters:
        # the string to be hashed, and the parameter of the hash, which will be
        # just the no. of it now
        self.__hash = hashf
        
        # storing the number of hash functions
        self.__k = hashfk
        
    def __get_bitz(self, item):
        # calls the hash function the given "k" times, returns the hash values in a list
        return [self.__hash(item, kc) for kc in range(self.__k)]
    
    def add(self, item):
        try:
            numpy.put(self.__bitarray, self.__get_bitz(item), 1)
        except IndexError:
            print 'That offset isnt here!'
        
    def lookup(self, item):
        # get the set bit (indexes) for the given item
        bitz = self.__get_bitz(item)
        
        # check it against the stored bitstring
        # returns False if not all of the items are nonzero
        return numpy.count_nonzero(numpy.take(self.__bitarray, bitz)) == len(bitz)        

The Bloom filter class shown above is quite general - due to the fact, that many of the parameters are up to the user.

The desired size of the filter (*m*), the hash function and the number of hash functions (*k*) are specified when instantiating the class. According to the exercise quite, this time we will use a bitstring with 1000000 (one-million) bits, and the number of the hash functions are calculated as: $m/n * ln(2)$. The *n* in the equation is the expected number of distinct elements in the input, equals with the length of our English dictionary in lines.

In [47]:
import math # due to log() and ceil()

m = 1000000
n = sum(1 for line in open('dict'))

k = int(math.ceil(float(m) / n  * math.log(2)))

print 'The exact values then: m = %d, n = %d, k = %d' % (m, n, k)

The exact values then: m = 1000000, n = 235886, k = 3


The hash function is the **MurmurHash3** this time, which is just passed to the class as a lambda function. Its second parameter (the index of the given hash function in the Bloom filter) will be used as a seed to the hash function, yielding a different (albeit of course reproduceable) hash for the same input string.

In [48]:
import mmh3

# using the MurmurHash3, and the number of the hash function as the seed
# it is divided by % in order to limit the results to the 'm' range
hash_function = lambda x, y : mmh3.hash(x, y) % m

Then, the class can be created finally:

In [49]:
filter = bloom_filter(m, hash_function, k)

The Shakespeare-text was processed using the following Unix command:

`tr -s  '[:blank:]\n' '\n' < shakespeare.txt | tr 'A-Z' 'a-z' | tr -cd [:lower:][:cntrl:] | sed '/^$/d' | sort | uniq > shakespeare_words.txt`

in order to get a file containing only the separate, unique words.

Then, the Bloom filter is filled with all the words from *dict*:

In [50]:
for line in open('dict'):
    filter.add(line)

In [44]:
blahblah

NameError: name 'blahblah' is not defined

In [51]:
not_in_filter = []
for word in open('shakespeare_words.txt'):
    if not filter.lookup(word):
        not_in_filter
        print word, ' is probably not in the filter.'

abandond
 is probably not in the filter.
abandonwhich
 is probably not in the filter.
abused
 is probably not in the filter.
abuses
 is probably not in the filter.
acres
 is probably not in the filter.
actions
 is probably not in the filter.
acts
 is probably not in the filter.
adam
 is probably not in the filter.
addressd
 is probably not in the filter.
affairs
 is probably not in the filter.
ages
 is probably not in the filter.
aliena
 is probably not in the filter.
allottery
 is probably not in the filter.
allows
 is probably not in the filter.
alls
 is probably not in the filter.
ambles
 is probably not in the filter.
amiens
 is probably not in the filter.
anatomized
 is probably not in the filter.
andin
 is probably not in the filter.
animals
 is probably not in the filter.
answerd
 is probably not in the filter.
answered
 is probably not in the filter.
answers
 is probably not in the filter.
appears
 is probably not in the filter.
appointed
 is probably not in the filter.
approac